In [7]:
# Importing libraries
import datatable as dt
import pandas as pd
import altair as alt
from datatable import f,by,count,update,sort,join

In [8]:
dt.init_styles()
dt.options.display.head_nrows=4
dt.options.display.tail_nrows=4

In [9]:
policia_dt = dt.fread('https://assets.datacamp.com/production/repositories/1497/datasets/62bd9feef451860db02d26553613a299721882e8/police.csv',na_strings=[""])

In [10]:
weather_dt = dt.fread('https://assets.datacamp.com/production/repositories/1497/datasets/02f3fb2d4416d3f6626e1117688e0386784e8e55/weather.csv',na_strings=[""])

In [11]:
policia_dt

<Frame [91741 rows x 15 cols]>

In [12]:
weather_dt

<Frame [4017 rows x 27 cols]>

In [13]:
# count the number of missing values
policia_dt.countna()

<Frame [1 row x 15 cols]>

In [14]:
del policia_dt[:,['county_name', 'state']]

In [15]:
policia_dt

<Frame [91741 rows x 13 cols]>

In [17]:
policia_dt[:,count(),by(f.driver_gender)]

<Frame [3 rows x 2 cols]>

In [18]:
policia_tidy_dt = policia_dt[~dt.isna(f.driver_gender),:]

In [19]:
policia_tidy_df = policia_tidy_dt.to_pandas()

In [20]:
policia_tidy_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86536 entries, 0 to 86535
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   stop_date           86536 non-null  object
 1   stop_time           86536 non-null  object
 2   driver_gender       86536 non-null  object
 3   driver_race         86536 non-null  object
 4   violation_raw       86536 non-null  object
 5   violation           86536 non-null  object
 6   search_conducted    86536 non-null  bool  
 7   search_type         3307 non-null   object
 8   stop_outcome        86536 non-null  object
 9   is_arrested         86536 non-null  bool  
 10  stop_duration       86536 non-null  object
 11  drugs_related_stop  86536 non-null  bool  
 12  district            86536 non-null  object
dtypes: bool(3), object(10)
memory usage: 6.8+ MB


In [21]:
policia_tidy_df

stop_date stop_time driver_gender driver_race  \
0      2005-01-04     12:55             M       White   
1      2005-01-23     23:15             M       White   
2      2005-02-17     04:15             M       White   
3      2005-02-20     17:15             M       White   
4      2005-02-24     01:20             F       White   
...           ...       ...           ...         ...   
86531  2015-12-31     21:21             F       Black   
86532  2015-12-31     21:59             F       White   
86533  2015-12-31     22:04             M       White   
86534  2015-12-31     22:09             F    Hispanic   
86535  2015-12-31     22:47             M       White   

                        violation_raw            violation  search_conducted  \
0      Equipment/Inspection Violation            Equipment             False   
1                            Speeding             Speeding             False   
2                            Speeding             Speeding             False   
3                    Call for Service                Other             False   
4                            Speeding             Speeding             False   
...                               ...                  ...               ...   
86531         Other Traffic Violation     Moving violation             False   
86532                        Speeding             Speeding             False   
86533         Other Traffic Violation     Moving violation             False   
86534  Equipment/Inspection Violation            Equipment             False   
86535          Registration Violation  Registration/plates             False   

      search_type   stop_outcome  is_arrested stop_duration  \
0             NaN       Citation        False      0-15 Min   
1             NaN       Citation        False      0-15 Min   
2             NaN       Citation        False      0-15 Min   
3             NaN  Arrest Driver         True     16-30 Min   
4             NaN       Citation        False      0-15 Min   
...           ...            ...          ...           ...   
86531         NaN       Citation        False      0-15 Min   
86532         NaN       Citation        False      0-15 Min   
86533         NaN       Citation        False      0-15 Min   
86534         NaN        Warning        False      0-15 Min   
86535         NaN       Citation        False      0-15 Min   

       drugs_related_stop district  
0                   False  Zone X4  
1                   False  Zone K3  
2                   False  Zone X4  
3                   False  Zone X1  
4                   False  Zone X3  
...                   ...      ...  
86531               False  Zone K2  
86532               False  Zone K3  
86533               False  Zone X3  
86534               False  Zone K3  
86535               False  Zone X4  

[86536 rows x 13 columns]

In [23]:
policia_tidy_dt[:,count(),by(f.violation)
               ][:,f[:].extend({'grand_tot':dt.sum(f.count)})
                ][:,f[:].extend({'prop':f.count/f.grand_tot})
                 ][:,f[:].remove(f.grand_tot),sort(-f.prop)
                  ]

<Frame [6 rows x 3 cols]>

In [24]:
policia_tidy_dt[:,count(),by(f.driver_gender,f.violation)
               ][:,f[:].extend({'group_tot':dt.sum(f.count)}),by(f.driver_gender)
                ][:,f[:].extend({'prop':f.count/f.group_tot})
                 ][:,f[:].remove(f[1])]

<Frame [12 rows x 5 cols]>

In [25]:
policia_tidy_dt[f.violation=="Speeding",:
               ][:,count(),by(f.driver_gender,f.stop_outcome)
                ][:,f[:].extend({'group_total':dt.sum(f.count)}),by(f.driver_gender)
                 ][:,f[:].extend({'prop':f.count/f.group_total})
                  ][:,f[:].remove([f[1]])]

<Frame [12 rows x 5 cols]>

In [26]:
policia_tidy_dt[:,count(),by(f.search_conducted)
               ]

<Frame [2 rows x 2 cols]>

In [27]:
policia_tidy_dt[:,count(),by(f.driver_gender,f.search_conducted)
               ][:,f[:].extend({'group_total':dt.sum(f.count)}),by(f.driver_gender)
                ][:,f[:].extend({'prop':f.count/f.group_total})
                 ][:,f[:].remove(f[1])]

<Frame [4 rows x 5 cols]>

In [28]:
policia_tidy_dt[~dt.isna(f.search_type),:
               ][:,count(),by(f.search_type)
                ]

<Frame [25 rows x 2 cols]>

In [29]:
policia_tidy_dt[dt.f.search_type.re_match("[\w\s\W]+Frisk"),:
               ][:,count(),by(f.driver_gender)]

<Frame [2 rows x 2 cols]>

In [30]:
policia_tidy_dt[:,count(),by(f.stop_duration)]

<Frame [3 rows x 2 cols]>

In [35]:
policia_tidy_dt[:,count(),by(f.stop_duration,f.is_arrested)
               ][:,f[:].extend({'group_total':dt.sum(f.count)}),by(f.stop_duration)
                ][:,f[:].extend({'prop':f.count/f.group_total})
                 ][:,f[:].remove(f[1])]

<Frame [6 rows x 5 cols]>

In [38]:
policia_tidy_dt[:,count(),by(f.driver_race,f.is_arrested)]

<Frame [10 rows x 3 cols]>